In [1]:
import math
import networkx as nx
def distance(x,y):
    a = 0
    for i in xrange(len(x)):
        a += math.pow((x[i] - y[i]), 2)
    return math.sqrt(a)
f = open('/home/denis/Desktop/newfile.vtk', 'rt')
a = []
while True:
    a = f.readline().split()
    if a[0] == 'POINTS':
        break
point_set = []

#map - переводит массив string в массив с заданным типом
for i in xrange(int(a[1])):
    point_set.append(map(float,f.readline().split()))
graph = nx.Graph()    
digraph = nx.DiGraph()
for i in xrange(int(f.readline().split()[1])):
    p = map(int, f.readline().split())
    graph.add_node(p[1])
    graph.add_node(p[2])
    digraph.add_node(p[1])
    digraph.add_node(p[2])
    graph.add_edge(p[1], p[2], weight=distance(point_set[p[1]], point_set[p[2]]))
    digraph.add_edge(p[1],p[2], capacity=distance(point_set[p[1]], point_set[p[2]]))
    digraph.add_edge(p[2],p[1], capacity=distance(point_set[p[1]], point_set[p[2]]))
print 'hello'
f.close()

hello


In [2]:
min = [10000,10000,10000]
max = [0,0,0]
for point in point_set:
    for i in xrange(3):
        if point[i] < min[i]:
            min[i] = point[i]
        if point[i] > max[i]:
            max[i] = point[i]
print min, max

#Находим предположительно нужные нам начальную и конечную точки,
#но они не точные, их нет в графе
pre_src = [(max[0] - min[0])/2.0, 1036, min[2]]
pre_finish = [(max[0] - min[0])/2.0, 1036, max[2]]
print pre_src, pre_finish

[-1.934134e-07, 1035.7, -3.509751e-07] [14819.376, 1409.8, 15892.272]
[7409.688000096707, 1036, -3.509751e-07] [7409.688000096707, 1036, 15892.272]


In [3]:
src = [0,0,0]
fin = [0,0,0]
src_i = 0
fin_i = 0

#Находим подбором ближайшие к предполагаемым реальные точки графа
print pre_src, pre_finish
for i in xrange(len(point_set)):
    if distance(point_set[i], pre_src) < 291.5:
        src = point_set[i]
        src_i = i
    if distance(point_set[i], pre_finish) < 46.5:
        fin = point_set[i]
        fin_i = i
print src, fin, src_i, fin_i

[7409.688000096707, 1036, -3.509751e-07] [7409.688000096707, 1036, 15892.272]
[7376.1602, 1325.1002, 0.0] [7443.2158, 1067.6, 15892.272] 26956 52308


In [4]:
#Находим кратчайший путь
path = nx.shortest_path(graph,src_i,fin_i)

In [5]:
outf = open('out.vtk', 'w+t')
edges = [[0,0,0]]
index = 0
outf.write('{0}\n{1}\n{2}\n'.format('# vtk DataFile Version 2.0', 'Calculation ground2','ASCII\nDATASET POLYDATA'))
outf.write('POINTS {0} float\n'.format(len(point_set)))
for i in point_set:
    outf.write('{0} {1} {2}\n'.format(i[0], i[1], i[2]))
    
#path - набор вершин, учавствующие в минимальном пути
#edges - готовая для записи в втк строка ("2 35633 54688")
#Записываем текущую вершину и предыдущую из этого же массива
for i in path:
    edges.append([2, i, edges[index][1]])
    index+=1
outf.write('LINES {0} {1}\n'.format(len(edges) - 2, 3*(len(edges) - 2)))

#Начинаем с 2, потому что в массиве два не нужных ребра с нулями в начале, нужными, чтобы избежать обращения
#к несуществующим элементам
for i in range(2,len(edges)):
    outf.write('2 {0} {1}\n'.format(edges[i][1], edges[i][2]))
outf.close()


In [6]:
#Поток, flow_dict - потоки между каждой из точек попарно (словарь словарей)
#Алгоритм по умолчанию - "Проталкивания предпотока"
flow_value, flow_dict = nx.maximum_flow(digraph, src_i, fin_i)

In [31]:
outf = open('flow.vtk', 'w+t')
outf.write('{0}\n{1}\n{2}\n'.format('# vtk DataFile Version 2.0', 'Calculation ground2','ASCII\nDATASET POLYDATA'))
outf.write('POINTS {0} float\n'.format(len(point_set)))
for i in point_set:
    outf.write('{0} {1} {2}\n'.format(i[0], i[1], i[2]))

flow_p = []
for i in flow_dict.keys():
    for j in flow_dict[i].keys():
        if flow_dict[i][j] != 0:
            flow_p.append('2 {0} {1}\n'.format(i, j))

outf.write('LINES {0} {1}\n'.format(len(flow_p), 3*len(flow_p)))
for i in flow_p:
    outf.write(i)
outf.close()